In [ ]:
import logging
import time
import requests
from datetime import date ,timedelta,datetime
from fastapi import APIRouter, Depends
from fastapi_utils.cbv import cbv
from pydantic import BaseModel
from sqlalchemy import func, insert, select ,update
from sqlalchemy.ext.asyncio import AsyncSession
from db import get_db_krx
from libs.airi_krx_scraper.data import AiriKrxDataScraper
from models.models_krx import ListedIssue
# from user import Admin, get_jwt_user
# from utils.SlackBot import SlackBot
import pymysql


router = APIRouter()
logger = logging.getLogger('uvicorn')
# slack = SlackBot()








@cbv(router=router)
class KrxStockIssue:
  db:AsyncSession = Depends(get_db_krx)
  # user:Admin = Depends(get_jwt_user)
  scraper = AiriKrxDataScraper()


  class ListedIssue(BaseModel):
    #date:date
    lisin_code:str
    ticker_symbol:int
    ticker_name_full:str
    ticker_name:str
    ticker_name_eng:str
    listing_date:str
    market_type:str
    security_type:str
    company_category:str
    share_type:str
    par_value:int
    listed_share:int
    # maintain:int  # 새로 추가한 컬럼(종목의 생존유무-> 기본값은 true , 만약 , 사라지게 되면 false로 바뀌게 된다.)



  @router.get('/all',
    summary='전종목 기본정보',
    # response_model=list[ListedIssue]   ######### 오류를 발생하게 하는 나쁜아이이다. 이거의 역할을 찾아보자
  )
  async def get_issued_all(self, market_type):
    try:
      res = self.scraper.get_issued_all(market_type)
      return res
    except Exception:
      logger.exception('전종목 기본정보')
      # slack.send('#airi-ra-error', traceback.format_exc())

    return None


  # # class AllListedIssueParam(BaseModel):
  #   market_type : str = 'ALL'
  #   curr_date : str = datetime.now().strftime("%Y%m%d")

  @router.post('/all',
    summary='전종목 기본정보 업데이트',
    # response_model=int,
  )
  async def update_issued_all(self):
    try:
      res = await self.get_issued_all("ALL")
      # print(res)
      if res and len(res) > 0:
        await self.db.execute(
          insert(ListedIssue),
          res,
        )
        await self.db.commit()

      return len(res)
    except Exception:
      await self.db.rollback()
      logger.exception('전종목 기본정보 업데이트')
      # slack.send('#airi-ra-error', traceback.format_exc())

    return None

#===================================================================================


# 테스트 코드



  # @router.get('/newTestAllData')
  # async def get_issued_all(self, market_type:str='ALL'):
      
  #     res= self.scraper.get_issued_all(market_type)

  #     print(res, "이번엔 크롤링한 데이터에요!")

  #     return res


  # @router.get('/originTestAllData')
  # async def get_origin_data(self):

  #   dbog = pymysql.connect(host="10.100.1.191",port=13306,user="dp",passwd="dp_!23$",db="scrap_krx", charset='utf8')

  #   con= dbog.cursor()

  #   con.execute("SELECT * FROM listed_issue")

  #   rows = con.fetchall()

  #   original =[]

  #   for row in rows:
  #       original.append({
  #       #### 'date': datetime.today(),
  #       'isin_code': row[0],
  #       'ticker_symbol': row[1],
  #       'ticker_name_full': row[2],
  #       'ticker_name': row[3], 
  #       'ticker_name_eng': row[4],
  #       'market_type': row[5],
  #       'share_type': row[6], 
  #       'listing_date': row[7], 
  #       'listed_share': row[8], 
  #       'par_value': row[9],
  #       'company_category': row[10],
  #       'security_type': row[11],
  #       'maintain':bool(row[12])   #bool로 형변환 
  #     })

      
        



  #   print(original , "기존 데이터 다 가져왔어요!@!")

   #real ---------------------- 
  

  async def get_issued_all(self, market_type:str='ALL'):
      
      res= self.scraper.get_issued_all(market_type)

      # print(res, "이번엔 크롤링한 데이터에요!")

      # origin=self.get_origin_data()

      # origin

      return res


      #상장 함수

  async def new_data(self, original, new):
      # new =  await self.get_issued_all()      
      # original = await self.get_origin_data()  
 

      for n in new:
        found= False
        for o in original:
            if n['ticker_symbol'] == o.ticker_symbol:
                # print(n['ticker_symbol'])
                # print('.')
                found = True
                break
        if not found:
            n1 = ListedIssue(**n)
            self.db.add(n1)
            # await self.db.execute(f"INSERT INTO listed_issue VALUES ('{n['isin_code']}','{n['ticker_symbol']}','{n['ticker_name_full']}','{n['ticker_name']}','{n['ticker_name_eng']}','{n['market_type']}','{n['share_type']}','{n['listing_date']}','{n['listed_share']}','{n['par_value']}','{n['company_category']}','{n['security_type']}', True) ")
            # await self.db.commit()
            print(f"{n['ticker_symbol']}")
            
            print("NEW")

        
      print("*FINISH*")

      

      

  async def get_origin_data(self):
      try:
        q = select(ListedIssue)
        rs = await self.db.execute(q)
        return rs.scalars().all()

        

        ### 위 방법을 사용한다면 , 아래 처럼 하지 않더라도 데이터를 불러오고 사용할 수 있게 된다!!!


        dbog = pymysql.connect(host="10.100.1.191",port=13306,user="dp",passwd="dp_!23$",db="scrap_krx", charset='utf8')

        con= dbog.cursor()
        con.execute("SELECT * FROM listed_issue")
        rows = con.fetchall()
        #-----------------------------------------------------------------------------
        # 1번 방법
        # arr = []
        # for row in rows:
        #   arr.append(row[0])
        # print(arr)

        # 2번 방법
        row = list(map(lambda x : x[:] ,rows))

  

        # original =[]
        # for row in rows:
        #   original.append({      
        #   'isin_code': row[0],
        #   'ticker_symbol': row[1],
        #   'ticker_name_full': row[2],
        #   'ticker_name': row[3], 
        #   'ticker_name_eng': row[4],
        #   'market_type': row[5],
        #   'share_type': row[6], 
        #   'listing_date': row[7], 
        #   'listed_share': row[8], 
        #   'par_value': row[9],
        #   'company_category': row[10],
        #   'security_type': row[11],
        #   'maintain':bool(row[12])   #bool로 형변환 
        # })

        # print(original , "기존 데이터 다 가져왔어요!@!")
      
        return row
      except Exception as e:
        print(e)
      

      

  @router.get('/finalTestAllData')
  async def get_finaldata(self):

    print("*START*")
    try:
      new =  await self.get_issued_all()      
      original:list[ListedIssue] = await self.get_origin_data()  
 
      ## what?!

      # :list[ListedIssue]
      #n1 = ListedIssue(**n)
      for o in original:
          found= False
        
          for i, n in enumerate(new):
            if o.ticker_symbol == n['ticker_symbol']:
              n1 = ListedIssue(**n)
              o.ticker_name_full = n1.ticker_name_full
              o.ticker_name = n1.ticker_name
              o.ticker_name_eng = n1.ticker_name_eng
              o.market_type = n1.market_type
              o.share_type = n1.share_type
              o.listing_date = n1.listing_date
              o.listed_share = n1.listed_share
              o.par_value = n1.par_value
              o.company_category = n1.company_category
              o.security_type = n1.security_type
              o.maintain = n1.maintain
              # await self.db.execute(f"UPDATE listed_issue SET ticker_name_full = '{n['ticker_name_full']}', ticker_name = '{n['ticker_name']}' ,  market_type =  '{n['market_type']}' , share_type = '{n['share_type']} ', listing_date ='{n['listing_date']}' , listed_share = '{n['listed_share']}' , par_value = '{n['par_value']}',company_category = '{n['company_category']}',security_type = '{n['security_type']}',maintain = True where ticker_symbol  ='{o[1]}'")
              # # await self.db.commit()
              # # print(o[1])
              # # print('update')
              found = True
              break
                  
          if not found:
            print(o.ticker_symbol)
            o.maintain = False
            # await self.db.execute(f"UPDATE listed_issue SET maintain = False where ticker_symbol ='{o.ticker_symbol}'")
            # await self.db.commit()
            print("DROP")

          # if i % 100 == 0:
          #   await self.db.commit()

            


      last = await self.new_data(original, new)
              
      await self.db.commit()
        
       
            
            # n1 = ListedIssue(**n)
            # o.ticker_name_full = n1.ticker_name_full
            # o.ticker_name = n1.ticker_name
            # o.ticker_name_eng = n1.ticker_name_eng
            # o.market_type = n1.market_type
            # o.share_type = n1.share_type
            # o.listing_date = n1.listing_date
            # o.listed_share = n1.listed_share
            # o.par_value = n1.par_value
            # o.company_category = n1.company_category
            # o.security_type = n1.security_type
            # o.maintain = n1.maintain

            # await self.db.execute(f"UPDATE listed_issue SET ticker_name_full = '{n['ticker_name_full']}', ticker_name = '{n['ticker_name']}' , ticker_name_eng = '{n['ticker_name_eng']}' , market_type =  '{n['market_type']}' , share_type = '{n['share_type']} ', listing_date ='{n['listing_date']}' , listed_share = '{n['listed_share']}' , par_value = '{n['par_value']}' where ticker_symbol  ='{o[1]}'")
            # print(o[1])
            # await self.db.commit()
            # print('업데이트 되었습니다')

            # if o[1] != n['ticker_symbol']:
      #------------------------------------------------------------------------
      ##상장 코드

      # for n in new:
      #   found= False
      #   for o in original:
      #       if n['ticker_symbol'] == o[1]:
      #           print(n['ticker_symbol'])
      #           print('maintain')
      #           found = True
      #           break
      #   if not found:
      #       await self.db.execute(f"INSERT INTO (isin_code,ticker_symbol,ticker_name_full,ticker_name,ticker_name_eng,market_type,share_type,listing_date,listed_share,par_value,company_category,security_type,maintain) VALUES('{n['isin_code']}','{n['ticker_symbol']}','{n['ticker_name_full']}','{n['ticker_name']}','{n['ticker_name_eng']}','{n['ticker_name_full']}','{n['market_type']}','{n['share_type']}','{n['listing_date']}','{n['listed_share']}','{n['par_value']}','{n['company_category']}','{n['security_type']}','{n['maintain']}')WHERE ticker_symbol ='{n['ticker_symbol']}' ")
      #       await self.db.commit()
      #       print(n['ticker_symbol'])
      #       print("new")
      #----------------------------------------------------------------------------
      # for n in new:
      #   for o in original:
      #     if n['ticker_symbol']== o :
      #       continue

      #     else:

      #       await self.db.execute("INSERT INTO listed_issue")
      #       print("상장됬습니다")
      #     return o

      # return "finish"


    except Exception as e:
      print(f"error : {e.with_traceback()}")
      return None

      

    
    



      # if res['ticker_symbol'] == original['ticker_symbol']:
      #   await self.db.execute(
      #     insert(ListedIssue),
      #     # res,
          
      #     )
          
      #   await self.db.commit()

      #   print("업데이트됬어요")
    #   return 'ss'
    
    # except:

    #     await self.db.execute("UPDATE listed_issue SET maintain = False ")

    #     print("바꿧어요!")
    
    # return None

      



    




      
   
